# Polynomial chaos over database

In this basic example we are going to create a global approximation of a model response using functional chaos over a design of experiment.

You will need to specify the distribution of the input parameters.
If not known, statistical inference can be used to select a possible candidate, and fitting tests can validate such an hypothesis.

In [1]:
from __future__ import print_function
import openturns as ot

In [2]:
# Create a function R^n --> R^p
# For example R^4 --> R
myModel = ot.SymbolicFunction(['x1', 'x2', 'x3', 'x4'], ['1+x1*x2 + 2*x3^2+x4^4'])

# Create a distribution of dimension n
# for example n=3 with indpendent components
distribution = ot.ComposedDistribution(
    [ot.Normal(), ot.Uniform(), ot.Gamma(2.75, 1.0), ot.Beta(2.5, 3.5, -1.0, 2.0)])

In [3]:
# Prepare the input/output samples
sampleSize = 250
X = distribution.getSample(sampleSize)
Y = myModel(X)
dimension = X.getDimension()

In [4]:
# build the orthogonal basis
coll = [ot.StandardDistributionPolynomialFactory(distribution.getMarginal(i)) for i in range(dimension)]
enumerateFunction = ot.LinearEnumerateFunction(dimension)
productBasis = ot.OrthogonalProductPolynomialFactory(coll, enumerateFunction)

In [5]:
# create the algorithm
degree = 6
adaptiveStrategy = ot.FixedStrategy(
    productBasis, enumerateFunction.getStrataCumulatedCardinal(degree))
projectionStrategy = ot.LeastSquaresStrategy()
algo = ot.FunctionalChaosAlgorithm(X, Y, distribution, adaptiveStrategy, projectionStrategy)
algo.run()

In [6]:
# get the metamodel function
result = algo.getResult()
metamodel = result.getMetaModel()

class=Point name=Unnamed dimension=1 values=[5.45188e-14]

In [ ]:
# Print residuals
result.getResiduals()